In [18]:
F=GF(17)
R.<x>=F[]

#P=R.random_element(degree=7)
P=3*x^7+2*x^6+10*x^5+4*x^4+9*x^3+13*x^2+10*x+2

#RS码拓展了四倍
DOMAIN_SIZE=(P.degree()+1)*4 #32
#def split_polynomial(poly: R) -> tuple[R, R]:#版本不支持
def split_polynomial(poly):
    x = poly.parent().gen() #返回代数结构的生成元,这里返回多项式的变量x
    coeffs = poly.coefficients(sparse=False)
    deg = poly.degree()
    Pe = R(sum(coeffs[i] * x^(i//2) for i in range(0, deg + 1, 2)))#得到偶次项,sum将这些项构造起来 
    Po = R(sum(coeffs[i] * x^((i-1)//2) for i in range(1, deg + 1, 2)))
    return Pe, Po

#用随机alpha折叠(提交)多项式
def fold_polynomial(poly,alpha):#poly:R,alpha:F->R
    Pe,Po=split_polynomial(poly)
    Pf=Pe+alpha*Po
    return Pf

#在域上每一个点进行评估
def evaluate_polynomial(poly): #poly:R->list[F]
    return [poly(i) for i in range(DOMAIN_SIZE)] #0-31

#第一轮
evaluations1=evaluate_polynomial(P)

# alpha应该是随机的：`F.random_element()`
# 但为了测试和可重复性，我们将使用一个固定值
alpha1=F(11)
folded1=fold_polynomial(P,alpha1)
print(folded1) #degree=3

#第二轮
evaluation2=evaluate_polynomial(folded1)
alpha2=F(6)
folded2=fold_polynomial(folded1,alpha2)
print(folded2) #degree=1

#第三轮
evaluation3=evaluate_polynomial(folded2)
alpha3=F(13)
folded3=fold_polynomial(folded2,alpha3)
print(folded3) #degree=0

#次数为0，为常数多项式
assert(folded3.degree()==0)

# 现在进行验证

# 首先我们选取一个随机索引z
# 再次说明，为了举例，我将其硬编码了
z=F(8)

#得到P(z)和P(-z)
P_z=evaluations1[z] #f_1(z)这其实是折叠过后的了
P_z_minus=evaluations1[-z] # f_1(-z)

# 在接下来的各层中，我们会在每一步对索引进行平方运算
z2=z^2
P_z2=evaluation2[z2] # f_2(z^2)
P_z_minus2=evaluation2[-z2] # f_2(-z^2)

z4=z2^2
P_z4=evaluation3[z4]
P_z_minus4=evaluation3[-z4]

round1_check = (((alpha1 + z) / (2*z)) * P_z) + (((alpha1 - z) / (2*(-z))) * P_z_minus)
assert(round1_check==P_z2) 


round2_check = (((alpha2 + z2) / (2*z2)) * P_z2) + (((alpha2 - z2) / (2*(-z2))) * P_z_minus2)
assert(round2_check == P_z4)

round3_check = (((alpha3 + z4) / (2*z4)) * P_z4) + (((alpha3 - z4) / (2*(-z4))) * P_z_minus4)
# 在上一次检查中，我们验证它等于最后一个折叠多项式
# 这是一个常数，验证者知道这一点
assert(round3_check == folded3)

print("验证成功!")


x^3 + 12*x^2 + 10*x + 10
x + 2
15
验证成功!
